In [3]:
from bs4 import BeautifulSoup
import requests

In [4]:
url = 'https://attack.mitre.org/techniques/mobile/'
page = requests.get(url)

In [5]:
if page.status_code == 200:
    soup = BeautifulSoup(page.text,'html')
else:
    print("request failed")

In [6]:
table_data = soup.find("table")
table_body = table_data.find("tbody")

# Initialize a list to store the extracted data
result = []

# Extract data from each row of the table
for table_row in table_body.findAll("tr"):
    # Check if the row contains technique data or sub-technique data
    table_cols = table_row.findAll(["th", "td"])
    if len(table_cols) == 3:
        # If it's a technique row, extract data as usual
        result.append({
            "id": table_cols[0].text.strip(),
            "name": table_cols[1].text.strip(),
            "Description": table_cols[2].text.strip()
        })
    elif len(table_cols) == 2:
        # If it's a sub-technique row, append it to the previous technique
        result[-1]["sub-techniques"].append({
            "id": table_cols[0].text.strip(),
            "name": table_cols[1].text.strip()
        })

result

[{'id': 'T1626',
  'name': 'Abuse Elevation Control Mechanism',
  'Description': 'Adversaries may circumvent mechanisms designed to control elevated privileges to gain higher-level permissions. Most modern systems contain native elevation control mechanisms that are intended to limit privileges that a user can gain on a machine. Authorization has to be granted to specific users in order to perform tasks that are designated as higher risk. An adversary can use several methods to take advantage of built-in control mechanisms in order to escalate privileges on a system.'},
 {'id': 'T1517',
  'name': 'Access Notifications',
  'Description': 'Adversaries may collect data within notifications sent by the operating system or other applications. Notifications may contain sensitive data such as one-time authentication codes sent over SMS, email, or other mediums. In the case of Credential Access, adversaries may attempt to intercept one-time code sent to the device. Adversaries can also dismiss

In [7]:
import sqlite3

def insert_data_into_table(result):
    # Connect to SQLite database (or create it if it doesn't exist)
    conn = sqlite3.connect('RHYM(database).db')
    cursor = conn.cursor()

    # Create the table if it doesn't exist
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS mitre_TechniquesMobile (
            id TEXT,
            name TEXT,
            Description TEXT
        )
    ''')

    # Insert data into the table
    for item in result:
        id = item['id']
        name = item['name']
        description = item['Description']

        cursor.execute('''
            INSERT INTO mitre_TechniquesMobile (id, name, Description)
            VALUES (?, ?, ?)
        ''', (id, name, description))

    # Commit the transaction and close the connection
    conn.commit()
    conn.close()

In [8]:
insert_data_into_table(result)